# Open AI मॉडेल्सचे फाइन ट्यूनिंग

हा नोटबुक Open AI कडून दिलेल्या [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) दस्तऐवजातील सध्याच्या मार्गदर्शनावर आधारित आहे.

फाइन-ट्यूनिंग तुमच्या अनुप्रयोगासाठी फाउंडेशन मॉडेल्सची कार्यक्षमता सुधारते, जे विशिष्ट वापर प्रकरण किंवा परिस्थितीशी संबंधित अतिरिक्त डेटा आणि संदर्भासह त्याचे पुनःप्रशिक्षण करून केले जाते. लक्षात ठेवा की _few shot learning_ आणि _retrieval augmented generation_ सारख्या प्रॉम्प्ट इंजिनिअरिंग तंत्रांनी तुम्हाला संबंधित डेटासह डीफॉल्ट प्रॉम्प्ट सुधारण्याची परवानगी दिली आहे ज्यामुळे गुणवत्ता वाढते. तथापि, या पद्धतींना लक्षित फाउंडेशन मॉडेलच्या कमाल टोकन विंडो आकाराने मर्यादा असते.

फाइन-ट्यूनिंगसह, आपण प्रत्यक्षात आवश्यक डेटासह मॉडेलचे पुनःप्रशिक्षण करत आहोत (जेणेकरून आपण कमाल टोकन विंडोमध्ये बसणाऱ्या उदाहरणांपेक्षा खूप अधिक उदाहरणे वापरू शकतो) - आणि एक _कस्टम_ आवृत्ती तैनात करत आहोत जी आता इन्फरन्स वेळेस उदाहरणे प्रदान करण्याची गरज नाही. यामुळे केवळ आमच्या प्रॉम्प्ट डिझाइनची कार्यक्षमता सुधारते (आम्हाला टोकन विंडो इतर गोष्टींसाठी वापरण्याची अधिक लवचिकता मिळते) तर संभाव्यतः आमचे खर्चही कमी होतात (इन्फरन्स वेळेस मॉडेलला पाठवायच्या टोकन्सची संख्या कमी करून).

फाइन ट्यूनिंगचे ४ टप्पे आहेत:
1. प्रशिक्षण डेटा तयार करा आणि अपलोड करा.
1. फाइन-ट्यून केलेले मॉडेल मिळवण्यासाठी प्रशिक्षण नोकरी चालवा.
1. फाइन-ट्यून केलेले मॉडेल मूल्यांकन करा आणि गुणवत्तेसाठी पुनरावृत्ती करा.
1. समाधानी झाल्यावर फाइन-ट्यून केलेले मॉडेल इन्फरन्ससाठी तैनात करा.

लक्षात ठेवा की सर्व फाउंडेशन मॉडेल्स फाइन-ट्यूनिंगला समर्थन देत नाहीत - नवीनतम माहितीसाठी [OpenAI दस्तऐवज तपासा](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst). तुम्ही पूर्वी फाइन-ट्यून केलेले मॉडेल देखील फाइन-ट्यून करू शकता. या ट्युटोरियलमध्ये, आपण `gpt-35-turbo` या लक्षित फाउंडेशन मॉडेलचा वापर फाइन-ट्यूनिंगसाठी करू. 

---


### Step 1.1: आपला डेटासेट तयार करा

चला एक चॅटबॉट तयार करूया जो तुम्हाला घटकांच्या आवर्त सारणी समजून घेण्यास मदत करेल, एखाद्या घटकाबद्दल प्रश्नांची उत्तरे लिमरिकच्या स्वरूपात देऊन. या _सोप्या_ ट्युटोरियलमध्ये, आपण फक्त मॉडेलला प्रशिक्षण देण्यासाठी काही नमुना प्रतिसादांसह डेटासेट तयार करू. वास्तविक वापराच्या प्रकरणात, तुम्हाला खूप अधिक उदाहरणांसह डेटासेट तयार करावे लागेल. तुमच्या अनुप्रयोग क्षेत्रासाठी जर एखादा खुला डेटासेट उपलब्ध असेल, तर तुम्ही तो वापरू शकता आणि फाइन-ट्युनिंगसाठी त्याचा पुनर्रचना करू शकता.

आपण `gpt-35-turbo` वर लक्ष केंद्रित करत आहोत आणि एकच टर्न असलेला प्रतिसाद (चॅट पूर्णता) पाहत आहोत, त्यामुळे आपण [या सुचवलेल्या स्वरूपाचा](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) वापर करून उदाहरणे तयार करू शकतो जी OpenAI चॅट पूर्णता आवश्यकतांचे प्रतिबिंबित करतात. जर तुम्हाला बहु-टर्न संभाषणात्मक सामग्री अपेक्षित असेल, तर तुम्ही [बहु-टर्न उदाहरण स्वरूप](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) वापराल ज्यामध्ये `weight` पॅरामीटर असतो जो सूचित करतो की कोणते संदेश फाइन-ट्युनिंग प्रक्रियेत वापरले जावेत (किंवा नाही).

आपल्या ट्युटोरियलसाठी येथे आपण सोप्या एकट्या टर्नच्या स्वरूपाचा वापर करू. डेटा [jsonl स्वरूपात](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) आहे ज्यात प्रत्येक ओळीत 1 रेकॉर्ड असतो, प्रत्येक JSON-फॉर्मॅटेड ऑब्जेक्ट म्हणून सादर केलेला. खालील तुकडा 2 रेकॉर्ड्स नमुन्यादाखल दाखवतो - पूर्ण नमुना संचासाठी [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) पहा (10 उदाहरणे) जे आपण फाइन-ट्युनिंग ट्युटोरियलसाठी वापरणार आहोत. **टीप:** प्रत्येक रेकॉर्ड _नक्कीच_ एका ओळीत परिभाषित केला पाहिजे (सामान्य JSON फाईलप्रमाणे ओळींमध्ये विभागू नका)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

वास्तविक वापराच्या प्रकरणात चांगल्या निकालांसाठी तुम्हाला खूप मोठा उदाहरण संच आवश्यक असेल - त्यामध्ये प्रतिसादांच्या गुणवत्तेचा आणि फाइन-ट्युनिंगसाठी लागणाऱ्या वेळ/खर्च यांचा समतोल असतो. आपण एक लहान संच वापरत आहोत जेणेकरून फाइन-ट्युनिंग लवकर पूर्ण करू शकू आणि प्रक्रिया स्पष्ट करू शकू. अधिक जटिल फाइन-ट्युनिंग ट्युटोरियलसाठी [हा OpenAI कुकबुक उदाहरण](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) पहा.


---

### टप्पा 1.2 आपला डेटासेट अपलोड करा

डेटा अपलोड करण्यासाठी Files API वापरा [जसे येथे वर्णन केले आहे](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). लक्षात ठेवा की हा कोड चालवण्यासाठी, तुम्ही खालील टप्पे आधी पूर्ण केलेले असणे आवश्यक आहे:
 - `openai` Python पॅकेज इन्स्टॉल केलेले असावे (तुम्ही नवीनतम वैशिष्ट्यांसाठी >=0.28.0 आवृत्ती वापरत आहात याची खात्री करा)
 - `OPENAI_API_KEY` पर्यावरण चल तुमच्या OpenAI API कीसाठी सेट केलेले असावे
अधिक जाणून घेण्यासाठी, कोर्ससाठी दिलेला [सेटअप मार्गदर्शक](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) पहा.

आता, तुमच्या स्थानिक JSONL फाइलमधून अपलोडसाठी फाइल तयार करण्यासाठी कोड चालवा.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Step 2.1: SDK सह फाइन-ट्यूनिंग जॉब तयार करा


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### टप्पा 2.2: नोकरीची स्थिती तपासा

`client.fine_tuning.jobs` API सह तुम्ही काही गोष्टी करू शकता:
- `client.fine_tuning.jobs.list(limit=<n>)` - शेवटच्या n फाईन-ट्यूनिंग नोकऱ्यांची यादी करा
- `client.fine_tuning.jobs.retrieve(<job_id>)` - विशिष्ट फाईन-ट्यूनिंग नोकरीचे तपशील मिळवा
- `client.fine_tuning.jobs.cancel(<job_id>)` - फाईन-ट्यूनिंग नोकरी रद्द करा
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - नोकरीतील n पर्यंतच्या घटनांची यादी करा
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

प्रक्रियेचा पहिला टप्पा म्हणजे _ट्रेनिंग फाईलची पडताळणी_ करणे, जेणेकरून डेटा योग्य स्वरूपात आहे याची खात्री होईल.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### टप्पा 2.3: प्रगतीवर लक्ष ठेवण्यासाठी इव्हेंट ट्रॅक करा


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### टप्पा 2.4: OpenAI डॅशबोर्डमध्ये स्थिती पहा


तुम्ही OpenAI वेबसाइटला भेट देऊन आणि प्लॅटफॉर्मच्या _Fine-tuning_ विभागाचा शोध घेऊन देखील स्थिती पाहू शकता. हे तुम्हाला चालू नोकरीची स्थिती दाखवेल, तसेच मागील नोकरीच्या अंमलबजावणीच्या धावण्याचा इतिहास ट्रॅक करण्याची परवानगी देखील देईल. या स्क्रीनशॉटमध्ये, तुम्ही पाहू शकता की मागील अंमलबजावणी अयशस्वी झाली, आणि दुसरी धाव यशस्वी झाली. संदर्भासाठी, हे तेव्हा घडले जेव्हा पहिल्या धावने चुकीच्या स्वरूपातील रेकॉर्डसह JSON फाइल वापरली - एकदा दुरुस्त केल्यावर, दुसरी धाव यशस्वीपणे पूर्ण झाली आणि मॉडेल वापरासाठी उपलब्ध झाले.

![Fine-tuning job status](../../../../../translated_images/mr/fine-tuned-model-status.563271727bf7bfba.png)


आपण खालीलप्रमाणे दाखविल्याप्रमाणे व्हिज्युअल डॅशबोर्डमध्ये खाली स्क्रोल करून स्थिती संदेश आणि मेट्रिक्स देखील पाहू शकता:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/mr/fine-tuned-messages-panel.4ed0c2da5ea1313b.png) |  ![Metrics](../../../../../translated_images/mr/fine-tuned-metrics-panel.700d7e4995a65229.png)|


---

### टप्पा 3.1: आयडी मिळवा आणि कोडमध्ये फाईन-ट्यून केलेले मॉडेल तपासा


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### टप्पा 3.2: प्लेग्राउंडमध्ये फाइन-ट्यून केलेला मॉडेल लोड करा आणि चाचणी करा

आपण आता फाइन-ट्यून केलेला मॉडेल दोन मार्गांनी चाचणी करू शकता. प्रथम, आपण प्लेग्राउंडला भेट देऊ शकता आणि मॉडेल्स ड्रॉप-डाउन वापरून आपल्या नव्याने फाइन-ट्यून केलेल्या मॉडेलला पर्यायांमधून निवडू शकता. दुसरा पर्याय म्हणजे फाइन-ट्यूनिंग पॅनेलमध्ये दाखवलेला "प्लेग्राउंड" पर्याय वापरणे (वरील स्क्रीनशॉट पहा) जो खालील _तुलनात्मक_ दृश्य सुरू करतो जे फाउंडेशन आणि फाइन-ट्यून केलेल्या मॉडेल आवृत्त्या बाजूने बाजूने जलद मूल्यमापनासाठी दाखवते.

![Fine-tuning job status](../../../../../translated_images/mr/fine-tuned-playground-compare.56e06f0ad8922016.png)

फक्त आपल्या प्रशिक्षण डेटामध्ये वापरलेला सिस्टम संदर्भ भरा आणि आपला चाचणी प्रश्न द्या. आपण पाहाल की दोन्ही बाजू समान संदर्भ आणि प्रश्नाने अद्ययावत होतात. तुलना चालवा आणि आपण त्यांच्यातील आउटपुटमधील फरक पाहू शकता. _लक्षात घ्या की फाइन-ट्यून केलेला मॉडेल आपल्या उदाहरणांमध्ये दिलेल्या स्वरूपात प्रतिसाद कसा तयार करतो तर फाउंडेशन मॉडेल फक्त सिस्टम प्रॉम्प्टचे पालन करते_.

![Fine-tuning job status](../../../../../translated_images/mr/fine-tuned-playground-launch.5a26495c983c6350.png)

आपण पाहाल की तुलना प्रत्येक मॉडेलसाठी टोकन मोजणी आणि अनुमानासाठी लागलेला वेळ देखील प्रदान करते. **हा विशिष्ट उदाहरण एक सोपा आहे जो प्रक्रियेचे प्रदर्शन करण्यासाठी आहे पण प्रत्यक्षात कोणत्याही वास्तविक डेटासेट किंवा परिस्थितीचे प्रतिबिंब नाही**. आपण लक्षात घेऊ शकता की दोन्ही नमुन्यांमध्ये समान टोकन संख्या आहे (सिस्टम संदर्भ आणि वापरकर्ता प्रॉम्प्ट समान आहेत) आणि फाइन-ट्यून केलेला मॉडेल अनुमानासाठी अधिक वेळ घेतो (कस्टम मॉडेल).

वास्तविक परिस्थितींमध्ये, आपण अशा खेळणीच्या उदाहरणाचा वापर करणार नाही, तर वास्तविक डेटावर फाइन-ट्यूनिंग कराल (उदा., ग्राहक सेवेकरिता उत्पादन सूची) जिथे प्रतिसादाची गुणवत्ता अधिक स्पष्ट असेल. _त्या_ संदर्भात, फाउंडेशन मॉडेलसह समतुल्य प्रतिसाद गुणवत्ता मिळविण्यासाठी अधिक कस्टम प्रॉम्प्ट अभियांत्रिकी आवश्यक असेल ज्यामुळे टोकन वापर वाढेल आणि संभाव्यतः अनुमानासाठी लागणारा प्रक्रिया वेळही वाढेल. _हे प्रयत्न करण्यासाठी, OpenAI कुकबुकमधील फाइन-ट्यूनिंग उदाहरणे पहा आणि सुरुवात करा._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:
हा दस्तऐवज AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून अनुवादित केला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित अनुवादांमध्ये चुका किंवा अचूकतेची कमतरता असू शकते. मूळ दस्तऐवज त्याच्या स्थानिक भाषेत अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी अनुवाद शिफारसीय आहे. या अनुवादाच्या वापरामुळे उद्भवलेल्या कोणत्याही गैरसमजुती किंवा चुकीच्या अर्थलागी आम्ही जबाबदार नाही.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
